In [ ]:
from scipy import *
import numpy as np
from numpy.random import rand, uniform
import matplotlib.pyplot as plt
import matplotlib.animation
import PlanktonSignaling.basics as PS
import PlanktonSignaling.BlobDetection as BD
import PlanktonSignaling.Deposition as DP
import profile

%matplotlib notebook

#%load_ext Cython

Note here that $d_1 = \frac{\kappa \lambda_0}{v^2}$ and $d_2 = \frac{\beta}{\lambda_0}$. For the correct deposition functions, use linAtanDep2 and atanDep2 to include tanh.

In [ ]:
meshsize = 80
num = 1600
LL = 2
Swimmers = PS.Plankton(DP.linAtanDep2,N = meshsize,depMaxStr=8.0e-4,Const=3,L=LL,k=0.02,epsilon=1e-5,speed=0.3,
                    lambda0=.87,kappa=1e-3,beta=.9,depTransWidth=0.0001,depThreshold=0.003,dens=num/(LL**2))

def initial_conditions(x,y):
    return(0*x+0.003)

print(Swimmers.d1)
print(Swimmers.d2)

In [ ]:
Swimmers.SetIC(initial_conditions)

pos = [array([0.1,0.1])]

aggreg = [array([0,0])]
areaAve = [array([0,0])]

th = rand()*2*pi
vel = [array([cos(th),sin(th)])]

for l in range(0,40):
    for k in range(0,40):
        pos = np.append(pos,[array([mod(k*(Swimmers.L*0.05/2) + 0.01*(rand()-0.5) + (Swimmers.L*0.05/2),Swimmers.L),
                                  mod(l*(Swimmers.L*0.05/2) + 0.01*(rand()-0.5) + (Swimmers.L*0.05/2),Swimmers.L)])],axis=0)
        th  = rand()*2*pi
        vel = np.append(vel,[array([cos(th),sin(th)])],axis=0)

plt.figure()
plt.pcolormesh(Swimmers.xm,Swimmers.ym,Swimmers.Meshed())
plt.plot(pos[:,0],pos[:,1],'ro')
plt.colorbar()
plt.show()

In [ ]:
time = 100

pos_store = list([pos[:,:]])
pos_store = list([np.array(pos)])

scalar_store = list([Swimmers.Meshed()])
aggreg = np.array([0,0])

count = 1
for plot in range(0,1):
    for k in range(0,time):
        Swimmers.Update(Swimmers.scalar,pos,vel)
        pos_store.append(np.array(pos))
        scalar_store.append(Swimmers.Meshed())
        
        #Counts the Number of Aggregations at time k
        
        #aggreg = np.vstack((aggreg,[k+1,BD.countingAll(1,255,BD.Contour(Swimmers,pos,400,meshsize))]))  

# Animations

In [ ]:
fig   = plt.figure()
ax    = plt.subplot(1,1,1)
field = ax.pcolormesh(Swimmers.xm,Swimmers.ym,scalar_store[1])
field.set_clim(0,.0032)
dots, = ax.plot([], [], 'ro')
fig.colorbar(field)

def initit():
    dots.set_data([], [])
    return field,dots

def animate(k):
    arr = scalar_store[k]
    arr = arr[:-1, :-1]
    field.set_array(arr.ravel())
    plt.title(r'Time = ${0}$'.format(k*Swimmers.k), fontsize = 20)
    dots.set_data(pos_store[k][:,0],pos_store[k][:,1])

    return field,dots,
    
anim = matplotlib.animation.FuncAnimation(fig,animate,frames=range(0,len(scalar_store),2),
                                          interval=80,blit=False,repeat=False)

# Uncomment if you want to save it to a file.  Requires mencoder or ffmpeg or some writer utility to generate the file.
#anim.save('New_Movie_Long_10000.mp4', writer='ffmpeg')
plt.show()